# Musical Track Database
This application will read an iTunes export file in XML and produce a properly normalized database with this structure:

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);

If you run the program multiple times in testing or with different files, make sure to empty out the data before each run.  
  
You can use this code as a starting point for your application: http://www.py4e.com/code3/tracks.zip. The ZIP file contains the Library.xml file to be used for this assignment. You can export your own tracks from iTunes and create a database, but for the database that you turn in for this assignment, only use the Library.xml data that is provided.  

To grade this assignment, the program will run a query like this on your uploaded database and look for the data it expects to see:  

"""  
SELECT Track.title, Artist.name, Album.title, Genre.name   
    FROM Track JOIN Genre JOIN Album JOIN Artist   
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id   
        AND Album.artist_id = Artist.id  
    ORDER BY Artist.name LIMIT 3  
"""

In [9]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Genre;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

In [10]:
fname = "Library.xml"#input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key): #
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname) #save object
all = stuff.findall('dict/dict/dict') #extract inside dict
print('Dict count:', len(all))

Dict count: 404


In [18]:
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    genre = lookup(entry, 'Genre') #add
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None : 
        continue

    print(name, artist, genre, album, count, rating, length)

    #add new data to each table
    #1. add artist_id
    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0] #search artist_id and if it is new, add artist_id
    
    #2. add album_id
    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    #3. add genre_id
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]
    
    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    conn.commit()

Another One Bites The Dust Queen Rock Greatest Hits 55 100 217103
Asche Zu Asche Rammstein Industrial Herzeleid 79 100 231810
Beauty School Dropout Various Soundtrack Grease 48 100 239960
Black Dog Led Zeppelin Rock IV 109 100 296620
Bring The Boys Back Home Pink Floyd Rock The Wall [Disc 2] 33 100 87118
Circles Bryan Lee Funk Blues Is 54 60 355369
Comfortably Numb Pink Floyd Rock The Wall [Disc 2] 36 100 384130
Crazy Little Thing Called Love Queen Rock Greatest Hits 38 100 163631
Electric Funeral Black Sabbath Metal Paranoid 44 100 293015
Fat Bottomed Girls Queen Rock Greatest Hits 38 100 257515
For Those About To Rock (We Salute You) AC/DC Rock Who Made Who 84 100 353750
Four Sticks Led Zeppelin Rock IV 84 100 284421
Furious Angels Rob Dougan Soundtrack The Matrix Reloaded 54 100 330004
Gelle Bryan Lee Blues/R&B Blues Is 45 60 199836
Going To California Led Zeppelin Rock IV 100 100 215666
Grease Various Soundtrack Grease 42 100 205792
Hand of Doom Black Sabbath Metal Paranoid 36 100 

Heat Dies Down Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 12 None 237061
Highroyds Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 10 None 199575
Love's Not A Competition (But I'm Winning) Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 11 None 197799
Thank You Very Much Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 11 None 157753
I Can Do It Without You Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 11 None 204199
My Kind Of Guy Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 10 None 246595
Everything Is Average Nowadays Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 16 None 164754
Learnt My Lesson Well Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 9 None 325955
Try Your Best Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 8 None 222511
Retirement Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mob 10 None 237426
The Angry Mob [Live From Berlin] Kaiser Chiefs Alternative & Punk Yours Truly, Angry Mo

TypeError: 'NoneType' object is not subscriptable